In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
file_path = Path('../../Dados/Dados-Metereologicos/Horarios/' +\
     'dados_82798_H_C_2008-01-01_2013-12-31.csv'
     )

In [6]:
#importando o dataframe a partir do csv dos dados horários
df_horario = pd.read_csv(file_path, header=9, sep=';')

#excluindo a coluna a mais
df_horario.drop('Unnamed: 6', axis=1, inplace=True)

#tornando a hora de medição um número do tipo horas:00
df_horario['Hora Medicao'] = (df_horario['Hora Medicao']//100).astype(str)+':'+'00' 

#concatenando a hora com a data de medição e retirando a coluna das horas
df_horario['Data Medicao'] = df_horario['Data Medicao'] + ' ' + df_horario['Hora Medicao'].astype(str)
df_horario.drop('Hora Medicao', axis=1, inplace=True)

#transformando a coluna Data Medicao para o tipo datetime
df_horario['Data Medicao'] = pd.to_datetime(df_horario['Data Medicao'])

#tornando a coluna em índices.
df_horario.set_index('Data Medicao', inplace=True)

#Por fim, transformando os dados para o tipo float
for coluna in df_horario.columns:
    df_horario[coluna] = df_horario[coluna].str.replace(',','.').astype(float)

df_horario.head()

,"PRECIPITACAO TOTAL, HORARIO(mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)","TEMPERATURA DO AR - BULBO UMIDO, HORARIA(°C)","VENTO, VELOCIDADE HORARIA(m/s)"
Data Medicao,,,,
2008-01-01 00:00:00,0.0,26.8,22.6,3.3
2008-01-01 12:00:00,0.0,29.4,23.3,3.5
2008-01-01 18:00:00,0.0,29.6,23.5,3.3
2008-01-02 00:00:00,0.0,26.6,22.2,2.6
2008-01-02 12:00:00,0.0,29.8,23.9,3.3
